In [ ]:
import numpy as np

from helpers import pre_processing_wlan_utils as preprocess_utils
from helpers import classifier_wlan_spectral_utils as classifier_utils

In [ ]:
#Set which task you want to solve. This value will be used to select the right labels for the loaded dataset.
#The three task can be 'phy', 'frames', 'app-type', or, 'app'
task = 'app'
label = preprocess_utils.label_index[task]
num_classes = preprocess_utils.num_classes[task]
labels_string = preprocess_utils.labels_string[task]
print("Label id: ", label)
print("Num classes in that label: ", num_classes)
print("Labels: ", labels_string)

In [ ]:
#This dataset does not contain the unknown label class (L2 frames that were generated with an unknown app), so lets remove that label.
if (task == 'app') or (task == 'app-type'):
    num_classes = num_classes-1
    labels_string = labels_string[0:num_classes]
    print("Label id: ", label)
    print("Num classes in that label: ", num_classes)
    print("Labels: ", labels_string)

In [ ]:
#Set lenght of the sequences
seq_length = 3000

#Set type of padding. 
padding = 'post'

#Set path to dataset folder. All the files from the dataset can be downloaded from https://zenodo.org/record/5208201
dataset_folder = '../../dataset/waveforms/'

#Set name of dataset file. In this case we are using once of the balanced dataset (filename_balanced.mat)
dataset_filename = 'waveforms_2G_n_SNR_'+task+'_balanced.mat'

In [ ]:
#Get X and Y without padding/truncation nor scaling. The function get_raw_xy_spectrum returns the L1 packets (IQ samples) and all the labels associated to them. 
Xraw, Yraw = classifier_utils.get_raw_xy_spectrum(dataset_folder,dataset_filename)

In [ ]:
#Let's pad/truncate the L1 packets to a given lenght.
print("Padding/Truncating sequence to a length of ",str(seq_length))
X = classifier_utils.pad_or_trunc_x_and_scale(Xraw, seq_length, padding, scale=False)

In [ ]:
#Lets select the labels of the classification task and generate the one-shot labels 
print("Generate one-shot labels")
Y = classifier_utils.get_one_hot_labels(Yraw, num_classes, label)

In [ ]:
#Set a seed for pseudo random generator
seed = 42
print("Performing data splitting")
X_train, X_val, X_test, Y_train, Y_val, Y_test = classifier_utils.get_xy_4_training(X,Y,seed)
print(X_train.shape,X_val.shape, X_test.shape)

In [ ]:
#If you want to get not the L1 but L2 packets, simple use the following function 
Xraw_bytes, Yraw_bytes = classifier_utils.get_raw_xy_bytes(dataset_folder,dataset_filename)

In [ ]:
#Let's pad/truncate the L2 packets to the max length of a L2 packet found in the dataset
l2_seq_length = classifier_utils.get_max_length_L2_packet(Xraw_bytes)
print("The max length of a L2 packet was ",l2_seq_length) 

In [ ]:
print("Padding/Truncating sequence to a length of ",str(l2_seq_length))
X_byte = classifier_utils.pad_or_trunc_x_and_scale_bytes(Xraw_bytes, l2_seq_length, padding, scale=False)

In [ ]:
#Lets select the labels of the classification task and generate the one-shot labels 
print("Generate one-shot labels")
Y_byte = classifier_utils.get_one_hot_labels(Yraw_bytes, num_classes, label)

In [ ]:
#As before, let's a seed for pseudo random generator and split the dataset
seed = 42
print("Performing data splitting")
X_byte_train, X_byte_val, X_byte_test, Y_byte_train, Y_byte_val, Y_byte_test = classifier_utils.get_xy_4_training(X_byte,Y_byte,seed)
print(X_byte_train.shape,X_byte_val.shape, X_byte_test.shape)